In [1]:
import pandas as pd
import time
import numpy as np

# Load the Excel file
file_path = "data.xlsx"  # Change this to the path of your Excel file
questions_df = pd.read_excel(file_path)
questions_df = questions_df.sample(frac=1)

# Initialize the score
score = 0

# List to keep track of asked questions
asked_questions = []

def quiz_game():
    global score,asked_questions
    # Initialize the score
    score = 0

    # List to keep track of asked questions
    asked_questions = []

    # Function to get a random question that hasn't been asked before
    def get_random_question():
        available_questions = questions_df[~questions_df["Questions"].isin(asked_questions)]
        if available_questions.empty:
            return None
        return available_questions.sample().iloc[0]

    # Record the start time of the quiz
    quiz_start_time = time.time()

    # Ask 10 questions
    quit_flag = False  # Add this flag

    for i in range(10):
        if quit_flag:  # check the flag at the start of each iteration
            break

        question = get_random_question()

        if question is None:
            print("No more available questions.")
            break
        

        print(f"Question {i + 1}: {question['Questions']}")
        asked_questions.append(question["Questions"])  # Add the question to the list of asked questions
        a=0
        while True:
            start_time = time.time()  # Start timer
            user_answer = input("Your answer: ")
            end_time = time.time()  # End timer
            time_taken = end_time - start_time  # Calculate time taken in seconds
            print(f"Time taken for this answer: {time_taken:.2f} seconds")

            correct_answer = question["Answer"]

            # Check if user wants to quit or retry
            if user_answer.strip().lower() == "quit":
                print(f"\nYou chose to quit! Your current score is: {score} out of {i+1} questions asked.")
                quit_flag = True  # Set the flag to True
                break  # This will break out of the inner while loop
            elif user_answer.strip().lower() == "r":
                break  # Break out of the inner loop to get a new question
            
            if user_answer.strip() == correct_answer.strip():
                print("Correct!")
                if a ==0:
                    score += 1                
                    break
                break
            else:
                print(f"Sorry, the correct answer is: {correct_answer}")
                a =1
                

    # Record the end time of the quiz
    quiz_end_time = time.time()

    # Calculate total time taken for the quiz
    total_time_taken = quiz_end_time - quiz_start_time

    print(f"\nGame Over! Your final score is: {score} out of 10.")
    print(f"Total time taken for the quiz: {total_time_taken:.2f} seconds.")

# Run the quiz game
quiz_game()


Question 1: to join the words given in a list (my_list) separated by comma
Your answer: ",".join(my_list)
Time taken for this answer: 23.27 seconds
Correct!
Question 2: Select all columns excluding number datatypes
Your answer: pr
Time taken for this answer: 6.81 seconds
Sorry, the correct answer is: df.select_dtypes(exclude='number')
Your answer: df.selct_dtypes(exclude='number')
Time taken for this answer: 14.62 seconds
Sorry, the correct answer is: df.select_dtypes(exclude='number')
Your answer: df.select_dtypes(exclude='number')
Time taken for this answer: 22.18 seconds
Correct!
Question 3: You want to create a series of datetime objects with start date as start= '2023-01-01' and end date is end= '2023-12-01' , frequency- Daily
Your answer: date_range(start='2023-01-01',end='2023-12-01',freq='D')
Time taken for this answer: 48.18 seconds
Sorry, the correct answer is: pd.date_range(start=start,end=end,freq='D')
Your answer: pd.date_range(start='2023-01-01',end='2023-12-01',freq='D')